In [3]:
import sys
import hail as hl
import pandas as pd
import gnomad
from gnomad.resources.grch38 import gnomad
#from gnomad.resources.grch38 import gnomad
#import matplotlib.pyplot as plt # show() works!
import pandas as pd
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot

In [2]:
hl.init(driver_cores=8,
        worker_memory='standard',
        default_reference='GRCh38',
        tmp_dir='gs://pbn_pilot/proj-PBN/04_rare_SNPs/tmp/')


/opt/conda/miniconda3/lib/python3.11/site-packages/hail/context.py:354: UserWarning:

Using hl.init with a default_reference argument is deprecated. To set a default reference genome after initializing hail, call `hl.default_reference` with an argument to set the default reference genome.

/opt/conda/miniconda3/lib/python3.11/site-packages/hailtop/aiocloud/aiogoogle/user_config.py:43: UserWarning:

Reading spark-defaults.conf to determine GCS requester pays configuration. This is deprecated. Please use `hailctl config set gcs_requester_pays/project` and `hailctl config set gcs_requester_pays/buckets`.

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


SPARKMONITOR_LISTENER: Started SparkListener for Jupyter Notebook
SPARKMONITOR_LISTENER: Port obtained from environment: 34687
SPARKMONITOR_LISTENER: Application Started: application_1752605443147_0001 ...Start Time: 1752606194123


Running on Apache Spark version 3.5.0
SparkUI available at http://ychen-cnv-m.us-central1-a.c.pbn-analysis.internal:45779
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.135-034ef3e08116
LOGGING: writing to /home/hail/hail-20250715-1903-0.2.135-034ef3e08116.log


In [4]:
print("read in vcfs")
VCF_PATH = 'gs://fc-eb65ecd1-ad1b-4fed-a9cc-0f70571a9432/SSBC_BGE_pilot/*-SSBC-BGE-pilot.vcf.gz'
vcfs = [entry['path'] for entry in hl.hadoop_ls(VCF_PATH)]
mt = hl.import_vcf(vcfs, reference_genome='GRCh38', force_bgz=True, array_elements_required=False)

read in vcfs


In [5]:
print("Dataset pre-filtering")
print("filter multi alleles and split")
mt = hl.split_multi_hts(mt)

Dataset pre-filtering
filter multi alleles and split


In [6]:
mt = mt.repartition(2000, shuffle=False) 

In [7]:
print("vets(previously vqsr) filter")
mt = mt.filter_rows(mt.filters == hl.empty_set(hl.tstr), keep = True)

vets(previously vqsr) filter


In [8]:
print("1. LCRs filter")
#LCR_PATH="gs://soyeon-sga/resources/LCRFromHengHg38.bed"
LCR_PATH="gs://gcp-public-data--gnomad/resources/grch38/lcr_intervals/LCRFromHengHg38.txt"
lcr_intervals = hl.import_locus_intervals(LCR_PATH, reference_genome='GRCh38', skip_invalid_intervals=True)
mt = mt.filter_rows(hl.is_defined(lcr_intervals[mt.locus]), keep=False)

1. LCRs filter


2025-07-15 19:04:03.519 Hail: INFO: Reading table without type imputation1) / 1]
  Loading field 'f0' as type str (user-supplied)
  Loading field 'f1' as type int32 (user-supplied)
  Loading field 'f2' as type int32 (user-supplied)


In [9]:
ori=mt

In [10]:
print("2. Telomeres_and_Centromeres filter")
# cut -f1-3 hg38.telomeresAndMergedCentromeres.bed > hg38.telomeresAndMergedCentromeres_rmf4.bed
# gsutil -m cp hg38.telomeresAndMergedCentromeres_rmf4.bed gs://soyeon-sga/resources/
# Hail에서 GRCh38 기준으로 locus_interval()을 사용할 때 start 값이 1 이상이어야 하기 때문에 오류가 발생 0이면 1로 변환시킴
TeloCentro_INTERVALS="gs://pbn_pilot/proj-PBN/misc/hg38.telomeresAndMergedCentromeres.rmf4_0to1.bed"
telocentro_intervals = hl.import_locus_intervals(TeloCentro_INTERVALS, reference_genome="GRCh38", skip_invalid_intervals=True)
mt = mt.filter_rows(hl.is_defined(telocentro_intervals[mt.locus]), keep=False)
print(mt.count_rows())

2. Telomeres_and_Centromeres filter


2025-07-15 19:04:08.220 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (user-supplied)
  Loading field 'f1' as type int32 (user-supplied)
  Loading field 'f2' as type int32 (user-supplied)
2025-07-15 19:04:12.629 Hail: INFO: scanning VCF for sortedness...
2025-07-15 19:04:18.366 Hail: INFO: Coerced sorted VCF - no additional import work to do
2025-07-15 19:04:32.389 Hail: INFO: Coerced sorted dataset          (0 + 1) / 1]
2025-07-15 19:04:33.010 Hail: INFO: Ordering unsorted dataset with network shuffle


173918


In [11]:
print("3. Segmentation duplicate filter")
SEGDUP_PATH="gs://gcp-public-data--gnomad/resources/grch38/seg_dup_intervals/GRCh38_segdups.bed"
segdup_intervals = hl.import_locus_intervals(SEGDUP_PATH, reference_genome='GRCh38', skip_invalid_intervals=True)
mt = mt.filter_rows(hl.is_defined(segdup_intervals[mt.locus]), keep=False)
print(mt.count_rows())

3. Segmentation duplicate filter


2025-07-15 19:05:24.625 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (user-supplied)
  Loading field 'f1' as type int32 (user-supplied)
  Loading field 'f2' as type int32 (user-supplied)
2025-07-15 19:05:39.282 Hail: INFO: Coerced sorted dataset
2025-07-15 19:05:39.647 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-15 19:05:40.139 Hail: INFO: Coerced sorted dataset


155108


In [12]:
print("TWIST Exome target region interval filter")
TARGET_INTERVALS="gs://pbn_pilot/proj-PBN/misc/Twist_Alliance_Clinical_Research_Exome_Covered_Targets_hg38-34.9MB.bed"
twist_intervals = hl.import_locus_intervals(TARGET_INTERVALS, reference_genome="GRCh38", skip_invalid_intervals=True)
mt = mt.filter_rows(hl.is_defined(twist_intervals[mt.locus]), keep=True)
print(mt.count_rows())

TWIST Exome target region interval filter


2025-07-15 19:06:46.315 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (user-supplied)
  Loading field 'f1' as type int32 (user-supplied)
  Loading field 'f2' as type int32 (user-supplied)
2025-07-15 19:07:00.143 Hail: INFO: Coerced sorted dataset
2025-07-15 19:07:00.573 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-15 19:07:01.005 Hail: INFO: Coerced sorted dataset
2025-07-15 19:07:01.914 Hail: INFO: Ordering unsorted dataset with network shuffle


155072


In [13]:
print("write Variant prefiltered mt")
mt = mt.checkpoint('gs://pbn_pilot/proj-PBN/04_rare_SNPs/00_raw_data/01_Var_PreFilterQC.SCZ_BGE_20250715.mt', overwrite=True)

write Variant prefiltered mt


2025-07-15 19:09:14.324 Hail: INFO: Coerced sorted dataset
2025-07-15 19:09:14.649 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-15 19:09:15.160 Hail: INFO: Coerced sorted dataset
2025-07-15 19:09:16.242 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-07-15 19:13:32.214 Hail: INFO: wrote matrix table with 155072 rows and 70 columns in 24 partitions to gs://pbn_pilot/proj-PBN/04_rare_SNPs/00_raw_data/01_Var_PreFilterQC.SCZ_BGE.mt
